# Presynaptic profiles analysis

---

Presynaptic axonal terminals with GCamp5f

In [ ]:
import sys
import os
import yaml

import matplotlib.pyplot as plt
import plotly.express as px

import numpy as np
import numpy.polynomial.polynomial as poly
import pandas as pd
import peakutils
import scikit_posthocs as sp

from scipy import signal
from scipy import stats
from scipy import ndimage

from skimage import io
from sklearn import preprocessing

## Full frame analysis

### Data & metadata uploading

#### Single data

In [ ]:
samp_name = 'A0011'
samp_path = os.path.join(''.join(sys.path[0].split('neuro')), 'data_neuro', samp_name)

# sample YAML metadata file uploading
with open(f'{samp_path}/{samp_name}_meta.yaml') as f:
    samp_meta = yaml.safe_load(f)

# sample img series uploading
img = io.imread(f'{samp_path}/{samp_name}_pre_mov_cor.tif')
plt.figure(figsize=(10,10))
plt.imshow(np.max(img, axis=0), cmap='jet')


# time parameters (from meta file)
total_reg_t = samp_meta['Reg_time']
frame_time = total_reg_t / img.shape[0]
time_line = np.linspace(0, total_reg_t, num=img.shape[0])

# treatment parameters (from meta file)
treatment_dict = {}
for samp in samp_meta['Events']:
    treat_name = f"{samp['Type']}_{samp['Freq']}"
    start_time = samp['Time']/1000
    end_time = start_time + samp['Duration']
    treatment_dict.update({treat_name:[start_time, end_time]})
for k,v in treatment_dict.items():  # app time print
    print(k, ':', np.around(v, 1))

application_lines_dict = {t:np.linspace(treatment_dict[t][0], treatment_dict[t][1]) for t in treatment_dict}


#### Multiple data

In [ ]:
class PreReg():
    """ Presynapse full-frame registration

    """
    def __init__(self, samp_name):
        self.samp_name = samp_name
        self.samp_path = os.path.join(''.join(sys.path[0].split('neuro')), 'data_neuro', samp_name)
        
        self.samp_plot_save_path = f'{self.samp_path}/pre_plot'
        if not os.path.exists(self.samp_plot_save_path):
             os.makedirs(self.samp_plot_save_path)

        self.img_series = io.imread(f'{self.samp_path}/{self.samp_name}_pre_mov_cor.tif')
        self.total_prof = np.asarray([np.mean(frame) for frame in self.img_series])

        with open(f'{self.samp_path}/{self.samp_name}_meta.yaml') as f:
            self.samp_meta = yaml.safe_load(f)

        self.total_reg_t = self.samp_meta['Reg_time']
        self.frame_time = self.total_reg_t / self.img_series.shape[0]
        self.time_line = np.linspace(0, self.total_reg_t, num=self.img_series.shape[0])

        print(f'{self.samp_name} ({self.samp_meta["Treat"]})')
        self.treat_dict, self.app_line_dict = self.treat_calc(self.samp_meta)


    @staticmethod
    def treat_calc(samp_meta):
        treat_dict = {}
        for samp in samp_meta['Events']:
            treat_name = f"{samp['Type']}_{samp['Freq']}"
            start_time = samp['Time']/1000
            end_time = start_time + samp['Duration']
            treat_dict.update({treat_name:[start_time, end_time]})
        for k,v in treat_dict.items():  # app time print
            print(k, ':', np.around(v, 1))

        application_lines_dict = {t:np.linspace(treat_dict[t][0], treat_dict[t][1]) \
                                  for t in treat_dict}
    
        return treat_dict, application_lines_dict
    

    def plot_tot_profile(self, save_debug_plot=False):
        plt.figure(figsize=(10, 4))
        plt.plot(self.time_line, self.total_prof)

        for line_name in self.app_line_dict:
                line_lim = self.app_line_dict[line_name]
                plt.plot(line_lim, [np.min(self.total_prof)] * len(line_lim),
                         label=line_name, linewidth=4)

        plt.xlabel('Time, s')
        plt.ylabel('I, a.u.')
        plt.title(f'PRE {self.samp_name} ({self.samp_meta["Treat"]}) FOV integral')    
        plt.tight_layout()    
        plt.legend()
        if save_debug_plot:
                plt.savefig(f'{self.samp_plot_save_path}/{self.samp_name}_pre_total_prof.png', dpi=300)
        else:
            plt.show()
        plt.close()


    def LF_stims(self, stim_ext_sec=15, LF_min_distance_sec=8, save_debug_plot=True):
        """ Extraction of low-frequency stimulation (0.1 Hz)

        """
        self.LF_min_distance_frames = int(LF_min_distance_sec / self.frame_time)
        if self.LF_min_distance_frames < 1:
            self.LF_min_distance_frames = 1

        self.LF_dF_stim_prof = {}
        self.LF_stim_df = pd.DataFrame(columns=['samp',
                                                'treat',
                                                'stim',
                                                'amp_raw',
                                                'amp_dF'])

        for l_n in self.treat_dict:
                if l_n in ['A+C_0.1', 'C_0.1']:
                        line_name = l_n
                        t_lim = self.treat_dict[line_name]
                        time_mask = (self.time_line>=(t_lim[0]-stim_ext_sec)) & \
                                    (self.time_line<=(t_lim[1]+stim_ext_sec))

                        stim_prof_raw = self.total_prof[time_mask]
                        stim_prof_base = peakutils.baseline(stim_prof_raw, 4)
                        stim_prof = (stim_prof_raw - stim_prof_base) \
                                     + np.mean(stim_prof_base)
                        stim_time = self.time_line[:len(stim_prof-3)]

                        F_0 = np.mean(stim_prof[:stim_ext_sec])
                        dF_stim_prof = (stim_prof-F_0)/F_0
                        dF_stim_prof_raw = (stim_prof_raw-np.mean(stim_prof_raw[:stim_ext_sec]))\
                                            / np.mean(stim_prof_raw[:10])
                        self.LF_dF_stim_prof.update({line_name:dF_stim_prof})

                        peaks, properties = signal.find_peaks(dF_stim_prof,
                                                              height=np.max(dF_stim_prof)*0.25,
                                                              distance=self.LF_min_distance_frames)
                        
                        stim_df = pd.DataFrame({'samp':np.full(len(peaks), self.samp_name),
                                                'treat':np.full(len(peaks), self.samp_meta["Treat"]), 
                                                'stim':np.full(len(peaks), line_name),
                                                'amp_raw':stim_prof[peaks],
                                                'amp_dF':dF_stim_prof[peaks]})
                        self.LF_stim_df = pd.concat([self.LF_stim_df, stim_df],
                                                    ignore_index=True)

                        plt.figure(figsize=(10, 4))
                        plt.plot(stim_time, dF_stim_prof,
                                    label='No base')
                        plt.plot(stim_time, dF_stim_prof_raw,
                                    label='Raw', linestyle='--', linewidth=0.75)
                        plt.plot(stim_time[peaks], dF_stim_prof[peaks], 'x',
                                    label='Peak', color='r')
                        plt.title(f'PRE {self.samp_name} ({self.samp_meta["Treat"]}), {line_name} ({len(peaks)} peaks)')
                        plt.xlabel('Time, s')
                        plt.ylabel('ΔF/F')
                        plt.legend()
                        plt.tight_layout()
                        if save_debug_plot:
                             plt.savefig(f'{self.samp_plot_save_path}/{self.samp_name}_{line_name}_pre_peaks_prof.png', dpi=300)
                        else:
                            plt.show()
                        plt.close()

                        if save_debug_plot:
                            # raw prof
                            plt.figure(figsize=(10, 4))
                            plt.plot(stim_time, stim_prof, label='raw no base')
                            plt.plot(stim_time, stim_prof_raw, label='raw', linestyle='--')
                            plt.plot(stim_time, stim_prof_base, label='baseline', linestyle=':')
                            plt.hlines(y=min(stim_prof_raw), xmin=0, xmax=stim_ext_sec-1,
                                    label='F0 win.', linewidth=4)
                            plt.title(f'PRE {self.samp_name} ({self.samp_meta["Treat"]}), {line_name}, raw baseline extraction')
                            plt.legend()
                            plt.xlabel('Time, s')
                            plt.ylabel('I, a.u.')
                            plt.tight_layout()
                            plt.savefig(f'{self.samp_plot_save_path}/{self.samp_name}_{line_name}_pre_raw_prof.png', dpi=300)
                            plt.close()

                            # # dF prof
                            # plt.figure(figsize=(10, 4))
                            # plt.plot(stim_time, dF_stim_prof, label='dF no base')
                            # plt.plot(stim_time, dF_stim_prof_raw, label='dF raw', linestyle='--')
                            # plt.hlines(y=min(dF_stim_prof_raw), xmin=0, xmax=stim_ext_sec-1,
                            #         label='F0 win.', linewidth=4)
                            # plt.title(f'{self.samp_name}, {line_name}, dF baseline extraction')
                            # plt.legend()
                            # plt.tight_layout()
                            # plt.savefig(f'{self.samp_plot_save_path}/{self.samp_name}_{line_name}_dF_prof.png', dpi=300)
                            # plt.close()


    def HF_stims(self, stim_ext_l=10, stim_ext_r=40, HF_min_distance_sec=8, save_debug_plot=True):
        """ Extraction of high-frequency stimulation (10 Hz)

        """
        self.HF_min_distance_frames = int(HF_min_distance_sec / self.frame_time)
        if self.HF_min_distance_frames < 1:
            self.HF_min_distance_frames = 1

        self.HF_dF_stim_prof = {}
        self.HF_stim_df = pd.DataFrame(columns=['samp',
                                                'treat',
                                                'stim',
                                                'amp_raw',
                                                'amp_dF',
                                                'FWHM',
                                                'AUC',
                                                'rise',
                                                'decay'])
        
        plt.figure(figsize=(7, 8))
        for l_n in self.treat_dict:
                if l_n in ['A+C_10.0', 'C_10.0', 'A_10.0']:
                        line_name = l_n
                        t_lim = self.treat_dict[line_name]
                        time_mask = (self.time_line>=(t_lim[0]-stim_ext_l)) & \
                                    (self.time_line<=(t_lim[1]+stim_ext_r))

                        stim_prof = self.total_prof[time_mask]
                        stim_time = self.time_line[:len(stim_prof)]

                        F_0 = np.mean(stim_prof[:stim_ext_l])
                        dF_stim_prof = (stim_prof-F_0)/F_0
                        self.HF_dF_stim_prof.update({line_name:dF_stim_prof})

                        peaks, prop = signal.find_peaks(dF_stim_prof,
                                                        height=np.max(dF_stim_prof)*0.25,
                                                        distance=self.HF_min_distance_frames,
                                                        prominence=(np.max(dF_stim_prof) \
                                                                    -np.min(dF_stim_prof))*0.5,
                                                        rel_height=0.98,
                                                        width=1)
                        fwhm_prop = signal.peak_widths(x=dF_stim_prof, peaks=peaks,
                                                       rel_height=0.5,
                                                       prominence_data=(prop['prominences'], prop['left_bases'], prop['right_bases']))
                        
                        prom = prop['prominences']
                        prom_h = dF_stim_prof[peaks] - prom
                        base_w = np.sort(np.asarray([*prop['left_ips'], \
                                                     *prop['right_ips']], dtype=int))
                        base_pairs =  np.asarray(list(zip(prop['left_ips'], \
                                                          prop['right_ips'])), dtype=int)
                        
                        fwhm_y = fwhm_prop[1]
                        fwhm_l = stim_time[np.asarray(fwhm_prop[2], dtype=int)]
                        fwhm_r = stim_time[np.asarray(fwhm_prop[3]+1, dtype=int)]

                        stim_df = pd.DataFrame({'samp':np.full(len(peaks), self.samp_name),
                                                'treat':np.full(len(peaks), self.samp_meta["Treat"]), 
                                                'stim':np.full(len(peaks), line_name),
                                                'amp_raw':stim_prof[peaks],
                                                'amp_dF':dF_stim_prof[peaks],
                                                'FWHM':fwhm_prop[1] * self.frame_time,
                                                'AUC':[np.sum(dF_stim_prof[bp[0]:bp[1]]) for bp in base_pairs],
                                                'rise':(peaks-prop['left_ips'])*self.frame_time,
                                                'decay':(prop['right_ips']-peaks)*self.frame_time})
                        self.HF_stim_df = pd.concat([self.HF_stim_df, stim_df],
                                                    ignore_index=True)

                        # peak with prop plotting
                        plt.vlines(x=stim_time[peaks],
                                   ymin=prom_h, ymax=dF_stim_prof[peaks], color='red')
                        plt.hlines(y=fwhm_y, xmin=fwhm_l, xmax=fwhm_r, color='red')
                        plt.plot(stim_time[peaks], dF_stim_prof[peaks], 'x', markersize=5, 
                                 color='r')
                        plt.plot(stim_time[base_w], dF_stim_prof[base_w], '.', markersize=10,
                                 color='red')
                        for peak_num in range(base_pairs.shape[0]):
                            base_width = base_pairs[peak_num]
                            peak_base_val = prop['width_heights'][peak_num]
                            plt.fill_between(x=stim_time[base_width[0]:base_width[1]], 
                                             y1=dF_stim_prof[base_width[0]:base_width[1]],
                                             y2=np.full((base_width[1]-base_width[0]), peak_base_val), 
                                             color="red",
                                             alpha=0.1)
                        plt.plot(stim_time, dF_stim_prof, label=line_name)

        plt.title(f'PRE {self.samp_name} ({self.samp_meta["Treat"]}), dF')
        plt.xlabel('Time, s')
        plt.ylabel('ΔF/F')
        plt.legend()
        plt.tight_layout()
        if save_debug_plot:
            plt.savefig(f'{self.samp_plot_save_path}/{self.samp_name}_{line_name}_pre_peaks_prof.png', dpi=300)
        else:
            plt.show()
        plt.close()
             

In [ ]:
ctrl_reg = PreReg('A0011')
ctrl_reg.plot_tot_profile(save_debug_plot=True)
ctrl_reg.LF_stims(stim_ext_sec=10, save_debug_plot=True)
ctrl_reg.HF_stims(stim_ext_l=25, stim_ext_r=30, save_debug_plot=True)


In [ ]:
ctrl5_reg = PreReg('A0005')
bicuc_reg = PreReg('A0012')
noglu_reg = PreReg('A0013')

In [ ]:
ctrl5_reg.plot_tot_profile(save_debug_plot=True)
ctrl5_reg.LF_stims(stim_ext_sec=10)
ctrl5_reg.HF_stims(stim_ext_l=25, stim_ext_r=30)

bicuc_reg.plot_tot_profile(save_debug_plot=True)
bicuc_reg.LF_stims(stim_ext_sec=10)
bicuc_reg.HF_stims(stim_ext_l=25, stim_ext_r=30)

noglu_reg.plot_tot_profile(save_debug_plot=True)
noglu_reg.LF_stims(stim_ext_sec=10)
noglu_reg.HF_stims(stim_ext_l=25, stim_ext_r=30)

## Stat

### DF combine

In [ ]:
LF_tot = pd.concat((ctrl_reg.LF_stim_df, bicuc_reg.LF_stim_df, noglu_reg.LF_stim_df), ignore_index=True)
HF_tot = pd.concat((ctrl_reg.HF_stim_df, bicuc_reg.HF_stim_df, noglu_reg.HF_stim_df), ignore_index=True)

In [ ]:
LF_tot

### LF stim. stat

#### Box-plot

In [ ]:
fig = px.box(LF_tot, x='treat' , y='amp_dF', color='treat', points='all', facet_col='stim')
fig.show()

#### A+C stim. HSD-test

In [ ]:
# https://blog.4dcu.be/programming/2021/12/30/Posthoc-Statannotations.html

group_0 = ctrl_reg.LF_stim_df['amp_dF'][ctrl_reg.LF_stim_df['stim'] == 'A+C_0.1']
group_1 = bicuc_reg.LF_stim_df['amp_dF'][bicuc_reg.LF_stim_df['stim'] == 'A+C_0.1']
group_2 = noglu_reg.LF_stim_df['amp_dF'][noglu_reg.LF_stim_df['stim'] == 'A+C_0.1']

ac_lf_krus_test = stats.kruskal(group_0, group_1, group_2)
print(ac_lf_krus_test)

ac_lf_dunn_test = sp.posthoc_dunn([group_0, group_1, group_2])
ac_lf_dunn_test


#### C stim. U-test

In [ ]:
c_lf_group_0 = ctrl_reg.LF_stim_df['amp_dF'][ctrl_reg.LF_stim_df['stim'] == 'C_0.1']
c_lf_group_1 = bicuc_reg.LF_stim_df['amp_dF'][bicuc_reg.LF_stim_df['stim'] == 'C_0.1']

c_lf_u_test = stats.mannwhitneyu(c_lf_group_0, c_lf_group_1)
print(c_lf_u_test)

### LF treat. stat

#### Box-plot

In [ ]:
fig = px.box(LF_tot, x='stim' , y='amp_dF', color='stim', points='all', facet_col='treat')
fig.show()

#### Ctrl U-test

In [ ]:
ctrl_lf_group_0 = ctrl_reg.LF_stim_df['amp_dF'][ctrl_reg.LF_stim_df['stim'] == 'C_0.1']
ctrl_lf_group_1 = ctrl_reg.LF_stim_df['amp_dF'][ctrl_reg.LF_stim_df['stim'] == 'A+C_0.1']

ctrl_lf_u_test = stats.mannwhitneyu(ctrl_lf_group_0, ctrl_lf_group_1)
print(ctrl_lf_u_test)

#### Bicuc. U-test

In [ ]:
ctrl_reg.LF_stim_df

In [ ]:
bicuc_lf_group_0 = bicuc_reg.LF_stim_df['amp_dF'][bicuc_reg.LF_stim_df['stim'] == 'C_0.1']
bicuc_lf_group_1 = bicuc_reg.LF_stim_df['amp_dF'][bicuc_reg.LF_stim_df['stim'] == 'A+C_0.1']

bicuc_lf_u_test = stats.mannwhitneyu(bicuc_lf_group_0, bicuc_lf_group_1)
print(bicuc_lf_u_test)

#### HF stat

In [ ]:
ctrl_group_1 = ctrl_reg.LF_stim_df['amp_dF'][ctrl_reg.LF_stim_df['stim'] == 'A+C_0.1']
ctrl_group_2 = ctrl_reg.LF_stim_df['amp_dF'][ctrl_reg.LF_stim_df['stim'] == 'C_0.1']

ctrl_u_val, ctrl_p_val = stats.mannwhitneyu(ctrl_group_1, ctrl_group_2)
print(f'U-test p-value={ctrl_p_val}')

fig = px.box(ctrl_reg.LF_stim_df, x='stim' , y='amp_dF', color='stim', points="all")
fig.show()

In [ ]:
bicuc_group_1 = bicuc_reg.LF_stim_df['amp_dF'][bicuc_reg.LF_stim_df['stim'] == 'A+C_0.1']
bicuc_group_2 = bicuc_reg.LF_stim_df['amp_dF'][bicuc_reg.LF_stim_df['stim'] == 'C_0.1']

bicuc_u_val, bicuc_p_val = stats.mannwhitneyu(bicuc_group_1, bicuc_group_2)
print(f'U-test p-value={bicuc_p_val}')

fig = px.box(bicuc_reg.LF_stim_df, x='stim' , y='amp_dF', color='stim', points="all")
fig.show()

In [ ]:
crop_up_df = LF_up_df[LF_up_df['cur'] >= 40].drop(['time', 'cur'], axis=1)
LF_total_df = pd.concat([LF_stim_df, crop_up_df], ignore_index=True)

group_0 = LF_total_df['amp_dF'][LF_total_df['stim'] == 'A+C_0.1']
group_1 = LF_total_df['amp_dF'][LF_total_df['stim'] == 'C_0.1']
group_2 = LF_total_df['amp_dF'][LF_total_df['stim'] == 'A+C_10-150_0.1']
hsd_test = stats.tukey_hsd(group_0, group_1, group_2)
print(hsd_test)

fig = px.box(LF_total_df, x='stim' , y='amp_dF', color='stim', points="all")
fig.show()